# Imports

In [0]:
#installation:
#!pip install googledrivedownloader

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
#imports:
import pandas as pd
import numpy as np
import math
from time import time

from sklearn.utils import shuffle
# from sklearn.preprocessing import PolynomialFeatures # unused here
# from sklearn.decomposition import TruncatedSVD # unused here

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

# Data upload

In [4]:
!mkdir data_files

import os.path
os.chdir('/content/data_files')

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1Vy6G1IFiWdTGoHo7YaNVYw7v9Xw3XnuT',
                                    dest_path='/content/data_files/Features_Variant_1.zip',
                                    unzip=True)

Unzipping...Done.


# Metrics

In [0]:
def custom_R2(target, pred):
    return 1-(np.sum((target-pred)**2))/(np.sum((target-np.mean(target))**2))

def custom_RMSE(target, pred, gamma, w):
    return np.sqrt(
        np.sum((target-pred)**2)/len(pred)+gamma*np.linalg.norm(w)
    )

# Preparation

## DataFrame Preparation

In [0]:
data=pd.read_csv('/content/data_files/Features_Variant_1.csv', header=None).drop_duplicates()

In [0]:
#get and fit features names
features_names=np.array(['page_likes_num',#1
               'page_checkins',#2
               'page_talking_about',#3
                'page_cat',#4
                'page_statistics',#5-29 #mean, avg etc.
                'comments_num_before_base_time',#30
                'comments_num_in_last_24_hours',#31 #last day
                'comments_num_in_last_48_to_24_hours',#32 #day before last
                'comments_num_in_first_24_hours',#33
                'comments_difference_in_last_two_days', #34 (32-31)
                'base_time', #35
                'character_num_in_post', #36
                'share_num',#37
                'post_promotion', #38 binary
                'h_local', #39 This describes the H hrs, for which we have the target variable/ comments received. 
                'post_published_weekday', #40-46 This represents the day(Sunday...Saturday) on which the post was published. 
                'base_datetime_weekday', #47-53 This represents the day(Sunday...Saturday) on selected base Date/Time. 
                'target' #54 The no of comments in next H hrs(H is given in Feature no 39).                
               ])

for index in range(5,29):
    features_names=np.insert(features_names, index, features_names[4]+'_'+str(index-4))
    
weekday=('sunday', 'monday','tuesday', 'wednesday', 'thursday', 'friday', 'saturday')    

for index in range(40,47):
    features_names=np.insert(features_names,index, features_names[39]+'_'+ weekday[index-40])
features_names=np.delete(features_names, 39)

for index in range(47,54):
    features_names=np.insert(features_names,index, features_names[46]+'_'+ weekday[index-47])
features_names=np.delete(features_names, 46)

data.columns=features_names

In [0]:
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative

## Data preparation

In [0]:
#categories:
data=data.drop(columns=['page_cat'])

data=pd.concat([data,pd.get_dummies(data['h_local'], prefix='h_local')], axis=1)
data=data.drop(columns=['h_local'])

In [0]:
#data shuffling
data=shuffle(data).reset_index(drop=True)

In [0]:
#new features:
data['comm_part_1']=data['comments_num_in_last_24_hours']/(data['comments_num_before_base_time']+1)
data['comm_part_4']=(data['comments_num_in_last_24_hours']-data['comments_num_in_last_48_to_24_hours'])/(data['comments_num_in_last_48_to_24_hours']+1)
data['comm_part_5']=(data['comments_num_in_last_24_hours']-data['comments_num_in_last_48_to_24_hours'])/(data['comments_num_before_base_time']+1)

#drop extra features:
data=data.drop(columns=[
                        'page_likes_num', 
                        'page_checkins', 
                        'page_statistics_10',
                        'page_statistics_23', 
                        'page_statistics_15',
                        'page_statistics_16',
                        'page_statistics_17',
                        'page_statistics_18',
                        'page_statistics_19',
                        'page_statistics_21',
                        'comments_num_in_first_24_hours',
       
       'character_num_in_post', 

       'post_published_weekday_sunday', 'post_published_weekday_monday',
       'post_published_weekday_tuesday', 'post_published_weekday_wednesday',
       'post_published_weekday_thursday', 'post_published_weekday_friday',
       'post_published_weekday_saturday', 

       'base_datetime_weekday_sunday',
       'base_datetime_weekday_monday', 'base_datetime_weekday_tuesday',
       'base_datetime_weekday_wednesday', 'base_datetime_weekday_thursday',
       'base_datetime_weekday_friday', 'base_datetime_weekday_saturday'])

# features transformation:
data['comments_num_in_last_48_to_24_hours']=data['comments_num_in_last_48_to_24_hours'].apply(lambda x: 0 if x==0 else 1)

# Model

In [0]:
# tree booster parameters:

# eta[default=0.3] in  0.01-0.2
# min_child_weight [default=1] 
# max_depth [default=6] Typical values: 3-10
# max_leaf_nodes
# gamma [default=0]
# max_delta_step [default=0]

# subsample [default=1] Typical values: 0.5-1
# colsample_bytree [default=1] Typical values: 0.5-1
# colsample_bylevel [default=1]

# lambda [default=1] L2 regularization term on weights - use it
# alpha [default=0] L1 regularization term on weight - use it i.e. 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05] (0.005 was chosen in example)

# scale_pos_weight [default=1] must be = 1 when there are a high disbalance


In [12]:
# grid search
# get some data
x_cols = data.columns.to_list()
x_cols.remove('target')
X, y = data.loc[:,x_cols].values, data.loc[:,'target'].values

# build a classifier
clf = xgboost.XGBRegressor(verbosity=0)

# use a full grid over 4 parameters
param_grid = {'max_depth': [3, 5, 10], 'min_child_weight': [1, 3, 6], 'alpha':[0.25,0.5,0.75], 'lambda':[0.25,0.5,0.75]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring="r2" ,iid=False)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))

res_grid=pd.DataFrame(grid_search.cv_results_)
res_grid['experiment_name']='grid search'

GridSearchCV took 3213.38 seconds for 81 candidate parameter settings.


In [13]:
# specify parameters and distributions
param_dist = {"max_depth": sp_randint(3, 10),
              "min_child_weight": sp_randint(1, 10),
              "alpha": uniform(loc=0, scale=0.1),
              "lambda": uniform(loc=0, scale=0.1)}


# run randomized search
n_iter_search = 81
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, scoring="r2",iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

res_rand=pd.DataFrame(random_search.cv_results_)
res_rand['experiment_name']='random search'

RandomizedSearchCV took 3128.28 seconds for 81 candidates parameter settings.


In [14]:
# run with default values
default = GridSearchCV(clf, param_grid={'max_depth': [6]}, cv=5, scoring="r2", iid=False)#default values
start = time()
default.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(default.cv_results_['params'])))

default=pd.DataFrame(default.cv_results_)
default['experiment_name']='default params'

GridSearchCV took 48.90 seconds for 1 candidate parameter settings.


In [15]:
# random groups test
#first group:
param_dist = {"max_depth": sp_randint(3, 10),
              "min_child_weight": sp_randint(1, 10)}

n_iter_search = 27
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV for first group took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

res_rand_1=pd.DataFrame(random_search.cv_results_)
res_rand_1['experiment_name']='random search for 1 group'        


#second group:
param_dist = {"alpha": uniform(loc=0, scale=0.1),
              "gamma":uniform(loc=0, scale=0.5),
              "lambda": uniform(loc=0, scale=0.1)}

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV for second group took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

res_rand_2=pd.DataFrame(random_search.cv_results_)
res_rand_2['experiment_name']='random search for 2 group' 


#third group:
param_dist = {"subsample":uniform(loc=0.5, scale=0.4),
              "colsample_bytree":uniform(loc=0.5, scale=0.4)}

random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, scoring="r2", iid=False)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV for third group took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

res_rand_3=pd.DataFrame(random_search.cv_results_)
res_rand_3['experiment_name']='random search for 3 group' 

RandomizedSearchCV for first group took 960.39 seconds for 27 candidates parameter settings.
RandomizedSearchCV for second group took 581.60 seconds for 27 candidates parameter settings.
RandomizedSearchCV for third group took 550.57 seconds for 27 candidates parameter settings.


In [0]:
res= pd.concat([res_grid, res_rand, res_rand_1,res_rand_2,res_rand_3,default])

In [17]:
res[:2]

,experiment_name,mean_fit_time,mean_score_time,mean_test_score,param_alpha,param_colsample_bytree,param_gamma,param_lambda,param_max_depth,param_min_child_weight,param_subsample,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
0,grid search,4.180195,0.022601,0.624615,0.25,NaN,NaN,0.25,3,1,NaN,"{'alpha': 0.25, 'lambda': 0.25, 'max_depth': 3...",46,0.756672,0.667498,0.562196,0.582756,0.553954,0.074679,0.000890,0.077356
1,grid search,4.165266,0.021516,0.629656,0.25,NaN,NaN,0.25,3,3,NaN,"{'alpha': 0.25, 'lambda': 0.25, 'max_depth': 3...",37,0.747616,0.669083,0.569692,0.577523,0.584365,0.019430,0.000585,0.069042


In [0]:
res.to_csv("res.csv",index=False)

In [0]:
from google.colab import drive 
drive.mount('/mntDrive') 
! ls "/mntDrive/My Drive"

In [0]:
cp res.csv "/mntDrive/My Drive/data/xgboost_perfomance.csv"